<a href="https://colab.research.google.com/github/FrankArcosAnselmo/mI_lab01_Fa/blob/develop/LAB04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SEMANA 4: NTEGRACIÓN Y LIMPIEZA. TABLA DE VARIABLES Y CREACIÓN DE NUEVAS VARIABLES. TRANSFORMACIÓN DE DATOS
# Alumno: Arcos Anselmo Frank Fernando Rodolfo

#A. Por medio de la librería ‘Pandas’, lea la base de datos, asigne nombres para las columnas, realice una imputación para la variable ‘Bare Nuclei’ por medio de la mediana y cambie las categorías para la variable ‘Class’ de 2 o 4 por 0 o 1.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#cargar datos del archivo

url= "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
df = pd.read_csv(url, header=None)
df.head()


,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [ ]:
# Nombres de columnas
columnas = ['ID', 'Clump_Thickness', 'Uniformity_Cell_Size', 'Uniformity_Cell_Shape',
            'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
            'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses', 'Class']


In [ ]:
df = pd.read_csv(url, names=columnas)


In [ ]:
# Reemplazar Class: 2 (benigno) por 0, 4 (maligno) por 1
df['Class'] = df['Class'].replace({2: 0, 4: 1})

In [ ]:
# Eliminar la columna ID porque no es predictiva
df.drop('ID', axis=1, inplace=True)

In [ ]:
print(df.head)

<bound method NDFrame.head of      Clump_Thickness  Uniformity_Cell_Size  Uniformity_Cell_Shape  \
0                  5                     1                      1   
1                  5                     4                      4   
2                  3                     1                      1   
3                  6                     8                      8   
4                  4                     1                      1   
..               ...                   ...                    ...   
694                3                     1                      1   
695                2                     1                      1   
696                5                    10                     10   
697                4                     8                      6   
698                4                     8                      8   

     Marginal_Adhesion  Single_Epithelial_Cell_Size  Bare_Nuclei  \
0                    1                            2          1.0   
1    

# B. Realice una detección de valores atípicos univariados por medio del método del rango intercuartílico con 3 de longitud a la derecha y 3 a la izquierda, y elimínelos. Además, realice una detección de valores atípicos multivariados por medio de las distancias de Mahalanobis y elimine aquellos valores que superen el valor de 30.

In [ ]:
df['Bare_Nuclei'].isna().sum()


np.int64(0)

In [ ]:
df['Bare_Nuclei'].isna().any()


np.False_

In [ ]:
# Rango intercuartílico con margen de 3
def eliminar_outliers_iqr(df, k=3):
    df_sin_outliers = df.copy()
    for col in df.columns[:-1]:  # Excluye 'Class'
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - k * IQR
        upper = Q3 + k * IQR
        df_sin_outliers = df_sin_outliers[(df_sin_outliers[col] >= lower) & (df_sin_outliers[col] <= upper)]
    return df_sin_outliers

df_iqr = eliminar_outliers_iqr(df)


In [ ]:
from scipy.spatial.distance import mahalanobis
from scipy.stats import zscore

# Calcular distancias de Mahalanobis
def eliminar_outliers_mahalanobis(df, threshold=30):
    df_temp = df.copy()
    X = df_temp.drop('Class', axis=1).values
    mean_vec = np.mean(X, axis=0)
    cov_matrix = np.cov(X, rowvar=False)
    inv_cov_matrix = np.linalg.pinv(cov_matrix)

    dists = [mahalanobis(x, mean_vec, inv_cov_matrix) for x in X]
    df_temp['Mahalanobis'] = dists

    # Filtrar por distancia
    df_filtrado = df_temp[df_temp['Mahalanobis'] <= threshold].drop('Mahalanobis', axis=1)
    return df_filtrado

df_maha = eliminar_outliers_mahalanobis(df_iqr)


#C. Realice un cálculo de los principales estadísticos descriptivos para las variables predictoras y cree dos conjuntos de datos: uno en donde se aplique la transformación Z-score y otro en donde se aplique la normalización mín-máx, sobre todas las variables predictoras.

In [ ]:
# Estadísticos descriptivos
estadisticos = df_maha.describe()
print(estadisticos)


       Clump_Thickness  Uniformity_Cell_Size  Uniformity_Cell_Shape  \
count       579.000000            579.000000             579.000000   
mean          3.851468              2.449050               2.578584   
std           2.528463              2.587315               2.550926   
min           1.000000              1.000000               1.000000   
25%           2.000000              1.000000               1.000000   
50%           3.000000              1.000000               1.000000   
75%           5.000000              3.000000               3.000000   
max          10.000000             10.000000              10.000000   

       Marginal_Adhesion  Single_Epithelial_Cell_Size  Bare_Nuclei  \
count         579.000000                    579.00000   579.000000   
mean            2.219344                      2.75475     2.718480   
std             2.315565                      1.80789     3.153268   
min             1.000000                      1.00000     1.000000   
25%       

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_zscore = scaler.fit_transform(df_maha.drop('Class', axis=1))

df_zscore = pd.DataFrame(X_zscore, columns=df_maha.columns[:-1])
df_zscore['Class'] = df_maha['Class'].reset_index(drop=True)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
X_minmax = minmax.fit_transform(df_maha.drop('Class', axis=1))

df_minmax = pd.DataFrame(X_minmax, columns=df_maha.columns[:-1])
df_minmax['Class'] = df_maha['Class'].reset_index(drop=True)
